<a href="https://colab.research.google.com/github/groneperrosato/academic/blob/main/Sistemas_de__recomendacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción

En este notebook vamos a dar nuestros primeros pasos en los sistemas de recomendación. Iremos avanzando poco a poco hasta crear un sistema de recomendación robusto que nos permita ejemplizar esta técnita.

Para ello, vamos autilizar un conjunto de datos de la plataforma [Movielens](https://movielens.org).

# Preparación del notebook y obtención de los datos

Después de habernos descargados los archivos, los guardamos en nuestro drive y activamos el acceso, así podremos acudir a ellos cada vez que lo necesitemos.

Como siempre, debemos de importar las librerías necesarias para poder desarrollar nuestro proyecto.

In [1]:
# importamos las librerias necesarias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# montamos el drive, para trabajar directamente con el

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# creamos nuestros datasets

peliculas = pd.read_csv("/content/drive/MyDrive/data/movies.csv",
                     names = ["peliculaId", "titulo", "generos"],
                     header = 0,
                     index_col = "peliculaId"
                    )

notas = pd.read_csv("/content/drive/MyDrive/data/ratings.csv",
                      names = ["usuarioId", "peliculaId", "nota", "momento"],
                      header = 0)

In [4]:
# separamos el año del nombre de la película

peliculas["anio"] = peliculas["titulo"].str[-5:-1]
peliculas["titulo"] = peliculas["titulo"].str[:-6]

In [5]:
# vemos un ejemplo de peliculas

peliculas.sample(5)

,titulo,generos,anio
peliculaId,,,
2625,Black Mask (Hak hap),Action|Adventure|Crime|Sci-Fi|Thriller,1996
85881,Win Win,Comedy|Drama,2011
104457,You're Next,Horror|Thriller,2011
239,"Goofy Movie, A",Animation|Children|Comedy|Romance,1995
6232,Born Free,Adventure|Children|Drama,1966


In [6]:
# vemos un ejemplo de notas

notas.head(5)

,usuarioId,peliculaId,nota,momento
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


# Sistemas de recomendación basados en popularidad

Como bien es sabido, existen dos tipo de sistemas de recomendación. El primer grupo y el que vamos a plasmar en este apartado, son los sistemas de recomendación por popularidad y son aquellos que se basan en popularidad de ciertos items para recomendarlos.

Este grupo consiste en recomendar aquellos items que son más demandados por los usuarios, o aquellos que están mejor valorados por los mismos, sin tener en cuenta las preferencias personales de cada uno. 

Están bien como una primer aproximación, pero se pueden mejorar mucho. Vamos a ver como sería.

## Primer intento: Sistemas de recomendación según número de visitas

En este primer intento, elaboraremos una lista, con las 15 películas más vistas. Para ello, supondremos que las veces que se ha puntuado una película, corresponde con las veces que ha sido vista la misma.

In [7]:
# agrupamos por película y vemos cuantas veces se han visualizado

peliculas["visualizaciones"] = notas["peliculaId"].value_counts()

In [8]:
# Ahora vemos qué pelicula se corresponde a la lista

peliculas.sort_values(by = "visualizaciones", ascending = False).head(15)

,titulo,generos,anio,visualizaciones
peliculaId,,,,
356,Forrest Gump,Comedy|Drama|Romance|War,1994,329.0
318,"Shawshank Redemption, The",Crime|Drama,1994,317.0
296,Pulp Fiction,Comedy|Crime|Drama|Thriller,1994,307.0
593,"Silence of the Lambs, The",Crime|Horror|Thriller,1991,279.0
2571,"Matrix, The",Action|Sci-Fi|Thriller,1999,278.0
260,Star Wars: Episode IV - A New Hope,Action|Adventure|Sci-Fi,1977,251.0
480,Jurassic Park,Action|Adventure|Sci-Fi|Thriller,1993,238.0
110,Braveheart,Action|Drama|War,1995,237.0
589,Terminator 2: Judgment Day,Action|Sci-Fi,1991,224.0


## Segundo intento: Sistema de recomendación según valoración de los usuarios

En este segundo caso, vamos a recomendar películas según la puntuación media que han recibido. Para ello, agruparemos por el campo de puntuación y luego tendremos en cuenta solo aquellas películas que hayan tenido un mínimo de 50 votos, para no considerar películas con muy pocas visualizaciones.

Seleccionaremos las 15 mejores películas que cumplan estos requisitos.

In [9]:
# agrupamos por notas medias

notas_medias = notas.groupby("peliculaId").mean()["nota"]

# añadimos las notas medias al conjunto de datos

peliculas["nota_media"] = notas_medias

In [10]:
peliculas

,titulo,generos,anio,visualizaciones,nota_media
peliculaId,,,,,
1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,215.0,3.920930
2,Jumanji,Adventure|Children|Fantasy,1995,110.0,3.431818
3,Grumpier Old Men,Comedy|Romance,1995,52.0,3.259615
4,Waiting to Exhale,Comedy|Drama|Romance,1995,7.0,2.357143
5,Father of the Bride Part II,Comedy,1995,49.0,3.071429
...,...,...,...,...,...
193581,Black Butler: Book of the Atlantic,Action|Animation|Comedy|Fantasy,2017,1.0,4.000000
193583,No Game No Life: Zero,Animation|Comedy|Fantasy,2017,1.0,3.500000
193585,Flint,Drama,2017,1.0,3.500000


In [11]:
# buscamos aquellas peliculas que se han visto mas de 50 veces y las ordenamos por nota media

peliculas.query("visualizaciones >= 50").sort_values(by = "nota_media", ascending = False).head(15)

,titulo,generos,anio,visualizaciones,nota_media
peliculaId,,,,,
318,"Shawshank Redemption, The",Crime|Drama,1994,317.0,4.429022
858,"Godfather, The",Crime|Drama,1972,192.0,4.289062
2959,Fight Club,Action|Crime|Drama|Thriller,1999,218.0,4.272936
1276,Cool Hand Luke,Drama,1967,57.0,4.271930
750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,1964,97.0,4.268041
904,Rear Window,Mystery|Thriller,1954,84.0,4.261905
1221,"Godfather: Part II, The",Crime|Drama,1974,129.0,4.259690
48516,"Departed, The",Crime|Drama|Thriller,2006,107.0,4.252336
1213,Goodfellas,Crime|Drama,1990,126.0,4.250000


In [12]:
# guardamos las películas con más de 50 visitas

peliculas_mas_50 = peliculas.query("visualizaciones >= 50")

peliculas_mas_50

,titulo,generos,anio,visualizaciones,nota_media
peliculaId,,,,,
1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,215.0,3.920930
2,Jumanji,Adventure|Children|Fantasy,1995,110.0,3.431818
3,Grumpier Old Men,Comedy|Romance,1995,52.0,3.259615
6,Heat,Action|Crime|Thriller,1995,102.0,3.946078
7,Sabrina,Comedy|Romance,1995,54.0,3.185185
...,...,...,...,...,...
109374,"Grand Budapest Hotel, The",Comedy|Drama,2014,52.0,3.778846
109487,Interstellar,Sci-Fi|IMAX,2014,73.0,3.993151
112852,Guardians of the Galaxy,Action|Adventure|Sci-Fi,2014,59.0,4.050847


# Sistemas de recomendación basados en gustos y preferencias

Ahora que ya hemos hecho el primer tipo de recomendadores, podemos ir un poco más allá y elaborar un sistema de recomendación algo más complejo, en el cual tengamos también en cuenta los gustos del usuario.

## Tercer intento

En este intento, recomendaremos a cada usuario una película según la última película que han visto.

Luego de haber identificado cual es la última película que vió, procederemos a recomendarle una lista de películas en función de la misma. Igual que en el caso anterior, tendremos en cuenta películas que hayan tenido más de 20 visualizaciones y las ordenaremos por el las notas generales.



In [13]:
# creamos una función la cual nos devuelva un top de peliculas según la última visualización de un usuario

def recomendador(usuario):

  print(f"Las recomendaciones para el usuario {usuario} son: \n")
  
  peliculas_vistas = notas.loc[notas["usuarioId"] == usuario]
  
  ultima_peli = peliculas_vistas.sort_values("momento", ascending = False).head(1)["peliculaId"].iloc[0]

  peliculas_sin_repetir = peliculas.drop(peliculas_usuarios[usuario])

  generos_ultima = peliculas.loc[ultima_peli]["generos"]

  pelis_similares = peliculas.loc[peliculas["generos"] == generos_ultima]

  pelis_recomendadas = pelis_similares.query("visualizaciones >=20").sort_values("nota_media", ascending = False).head(10)


  if pelis_recomendadas.empty:

    print("No hay ninguna película similar a la última visualizada por el usuario.")

  else:

    return pelis_recomendadas 

In [14]:
# creamos un diccionario con las películas que ha visto cada usuario

peliculas_usuarios = {}

for usuario in np.unique(notas["usuarioId"].values):

  peliculas_usuarios[usuario] = []

for usuario, pelicula in notas[["usuarioId", "peliculaId"]].values:
    peliculas_usuarios[usuario].append(pelicula)


In [20]:
# elegimos la recomendación para un usuario aleatorio

usuario_random = [np.random.randint(0, len(peliculas_usuarios))]

recomendador(usuario_random[0])

Las recomendaciones para el usuario 397 son: 



,titulo,generos,anio,visualizaciones,nota_media
peliculaId,,,,,
318,"Shawshank Redemption, The",Crime|Drama,1994,317.0,4.429022
858,"Godfather, The",Crime|Drama,1972,192.0,4.289062
1221,"Godfather: Part II, The",Crime|Drama,1974,129.0,4.259690
1213,Goodfellas,Crime|Drama,1990,126.0,4.250000
2329,American History X,Crime|Drama,1998,129.0,4.217054
1945,On the Waterfront,Crime|Drama,1954,24.0,4.187500
3147,"Green Mile, The",Crime|Drama,1999,111.0,4.148649
1084,Bonnie and Clyde,Crime|Drama,1967,35.0,4.071429
64614,Gran Torino,Crime|Drama,2008,46.0,3.945652


# Resolución Sprint 1

He hecho algunas cosas que no se recogen específicamente dentro del contenido de la clase, como la de crear una función que nos permita recomendar al usuario que queramos las películas, espero que no sea ningún impedimento.

Para la resolución del sprint 1, haremos algunas modificaciones en lo anteriormente descrito para devolver exactamente lo que se pide.

Como en este caso no se tiene el timestamp, se ha considerado la última película de la lista (4), como la última película que se ha visto.


In [21]:
# creación/modificación de una función para recomendar películas

def recomendador_sprint1(nombre, pelis):

  print(f"Las recomendaciones para {nombre} son: \n")
  
  peliculas_vistas = peliculas.loc[pelis]
  
  ultima_peli = pelis[-1]

  peliculas_sin_repetir = peliculas.drop(pelis)

  generos_ultima = peliculas.loc[ultima_peli]["generos"]

  pelis_similares = peliculas.loc[peliculas["generos"] == generos_ultima]

  pelis_recomendadas = pelis_similares.query("visualizaciones >= 50").sort_values("nota_media", ascending = False).head(10)


  if pelis_recomendadas.empty:

    print("No hay ninguna película similar a la última visualizada por el usuario.")

  else:

    return pelis_recomendadas 

In [24]:
# recomendación para Juan

pelis_juan = [2,10,296,2130,4]

recomendador_sprint1("Juan", pelis_juan)

Las recomendaciones para Juan son: 



,titulo,generos,anio,visualizaciones,nota_media
peliculaId,,,,,
1247,"Graduate, The",Comedy|Drama|Romance,1967,79.0,4.063291
6711,Lost in Translation,Comedy|Drama|Romance,2003,74.0,4.033784
6942,Love Actually,Comedy|Drama|Romance,2003,59.0,3.788136
2396,Shakespeare in Love,Comedy|Drama|Romance,1998,92.0,3.777174
56367,Juno,Comedy|Drama|Romance,2007,65.0,3.769231
1784,As Good as It Gets,Comedy|Drama|Romance,1997,96.0,3.697917
11,"American President, The",Comedy|Drama|Romance,1995,70.0,3.671429
3481,High Fidelity,Comedy|Drama|Romance,2000,75.0,3.666667
52973,Knocked Up,Comedy|Drama|Romance,2007,52.0,3.653846
